# Prétraitement des Données pour la Modélisation Électorale

**Objectif :** Ce notebook constitue la première étape du pipeline de machine learning. Il gère l'ensemble du processus de préparation des données :

1.  **Chargement des données brutes** depuis les fichiers CSV.
2.  **Création/Mise à jour des tables** dans une base de données PostgreSQL.
3.  **Création d'une vue unifiée (`elections_all`)** pour agréger toutes les données.
4.  **Prétraitement des données** (remplissage des valeurs manquantes, encodage, normalisation).
5.  **Sauvegarde des artefacts** (données traitées et transformateurs) pour les étapes suivantes.

## 1. Imports et Chargement de l'Environnement

In [84]:
import os
import joblib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from IPython.display import display

load_dotenv()
print("Librairies importées et variables d'environnement chargées.")

Librairies importées et variables d'environnement chargées.


## 2. Connexion à la Base de Données PostgreSQL

In [85]:
pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host = os.getenv("PG_HOST")
pg_port = os.getenv("PG_PORT")
pg_dbname = os.getenv("PG_DBNAME")

if not all([pg_user, pg_password, pg_host, pg_port, pg_dbname]):
    raise ValueError("Fichier .env incomplet. Vérifiez les variables PG_USER, PG_PASSWORD, etc.")

db_url = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}"
engine = create_engine(db_url)
print(f"Moteur de connexion à la base de données '{pg_dbname}' créé pour l'utilisateur '{pg_user}'.")

Moteur de connexion à la base de données 'ELECTIONS' créé pour l'utilisateur 'postgres'.


## 3. Pré-nettoyage : Suppression de l'Ancienne Vue

Pour pouvoir remplacer les tables de base, nous devons d'abord supprimer la vue `elections_all` qui en dépend. Elle sera recréée plus tard.

In [86]:
print("Suppression de la vue 'elections_all' pour permettre la mise à jour des tables...")
drop_view_query = "DROP VIEW IF EXISTS elections_all;"

try:
    with engine.connect() as connection:
        connection.execute(text(drop_view_query))
        connection.commit()
    print("✅ Vue 'elections_all' supprimée (si elle existait).")
except Exception as e:
    print(f"❌ Erreur lors de la suppression de la vue : {e}")

Suppression de la vue 'elections_all' pour permettre la mise à jour des tables...
✅ Vue 'elections_all' supprimée (si elle existait).


## 4. Chargement des Données depuis CSV et Création des Tables

In [87]:
DATA_DIR = '../data'

TABLE_CONFIG = {
    "unemployment_data": {
        "csv_file": "2017_2024_CHOMAGE_prepared.csv",
        "columns_map": {"ANNEE": "YEAR", "DEPARTEMENT_CODE": "DEPARTMENT_CODE", "DEPARTEMENT": "DEPARTMENT", "TX_CHOMAGE": "UNEMPLOYMENT_RATE"},
    },
    "crime_data": {
        "csv_file": "2017_2024_CRIMINALITE_prepared.csv",
        "columns_map": {"ANNEE": "YEAR", "DEPARTEMENT_CODE": "DEPARTMENT_CODE", "DEPARTEMENT": "DEPARTMENT", "NB_DE_VICTIMES": "NUMBER_OF_VICTIMS"},
    },
    "election_data": {
        "csv_file": "2017_2024_ELECTIONS_prepared.csv", 
        "columns_map": {"ANNEE": "YEAR", "DEPARTEMENT_CODE": "DEPARTMENT_CODE", "DEPARTEMENT": "DEPARTMENT", "GAGNANT": "WINNER", "NB_INSCRITS": "REGISTERED_VOTERS", "NB_VOTANTS": "VOTERS"},
    },
    "immigration_data": {
        "csv_file": "2017_2024_IMMIGRATION_prepared.csv",
        "columns_map": {"ANNEE": "YEAR", "DEPARTEMENT_CODE": "DEPARTMENT_CODE", "DEPARTEMENT": "DEPARTMENT", "TX_IMMIGRATION": "IMMIGRATION_RATE"},
    },
    "poverty_data": {
        "csv_file": "2017_2024_PAUVRETE_prepared.csv",
        "columns_map": {"ANNEE": "YEAR", "DEPARTEMENT_CODE": "DEPARTMENT_CODE", "DEPARTEMENT": "DEPARTMENT", "TX_PAUVRETE": "POVERTY_RATE"},
    },
}

print("Début du chargement des données CSV vers PostgreSQL...")
for table_name, config in TABLE_CONFIG.items():
    try:
        csv_path = os.path.join(DATA_DIR, config["csv_file"])
        df = pd.read_csv(csv_path, dtype={"DEPARTEMENT_CODE": str})
        df.rename(columns=config["columns_map"], inplace=True)
        if "DEPARTMENT_CODE" in df.columns:
            df["DEPARTMENT_CODE"] = df["DEPARTMENT_CODE"].astype(str).str.zfill(2)
        final_cols = [col for col in config["columns_map"].values() if col in df.columns]
        df_to_insert = df[final_cols]
        df_to_insert.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"✅ Table '{table_name}' chargée avec succès.")
    except Exception as e:
        print(f"❌ Erreur lors du chargement de la table '{table_name}': {e}")
print("\nChargement des tables de base terminé.")

Début du chargement des données CSV vers PostgreSQL...
✅ Table 'unemployment_data' chargée avec succès.
✅ Table 'crime_data' chargée avec succès.
✅ Table 'election_data' chargée avec succès.
✅ Table 'immigration_data' chargée avec succès.
✅ Table 'poverty_data' chargée avec succès.

Chargement des tables de base terminé.


## 5. Création de la Vue Agrégée `elections_all`

In [88]:
create_view_query = """
CREATE OR REPLACE VIEW elections_all AS
SELECT
    ed."YEAR",
    ed."DEPARTMENT_CODE",
    ed."WINNER",
    cd."NUMBER_OF_VICTIMS",
    id."IMMIGRATION_RATE",
    pd."POVERTY_RATE",
    ud."UNEMPLOYMENT_RATE"
FROM
    election_data ed
LEFT JOIN
    crime_data cd ON ed."DEPARTMENT_CODE" = cd."DEPARTMENT_CODE" AND ed."YEAR" = cd."YEAR"
LEFT JOIN
    immigration_data id ON ed."DEPARTMENT_CODE" = id."DEPARTMENT_CODE" AND ed."YEAR" = id."YEAR"
LEFT JOIN
    poverty_data pd ON ed."DEPARTMENT_CODE" = pd."DEPARTMENT_CODE" AND ed."YEAR" = pd."YEAR"
LEFT JOIN
    unemployment_data ud ON ed."DEPARTMENT_CODE" = ud."DEPARTMENT_CODE" AND ed."YEAR" = ud."YEAR";
"""

try:
    with engine.connect() as connection:
        connection.execute(text(create_view_query))
        connection.commit()
    print("✅ Vue 'elections_all' créée ou mise à jour avec succès.")
except Exception as e:
    print(f"❌ Erreur lors de la création de la vue: {e}")

✅ Vue 'elections_all' créée ou mise à jour avec succès.


## 6. Chargement et Préparation des Données (avec Imputation)

In [89]:
try:
    query = 'SELECT * FROM elections_all ORDER BY "DEPARTMENT_CODE", "YEAR";'
    df_full = pd.read_sql(query, engine)
    df_full.columns = df_full.columns.str.lower()

    print(f"Shape avant imputation : {df_full.shape}")
    df_full = df_full.sort_values(by=['department_code', 'year'])
    
    df_full = df_full.groupby('department_code', group_keys=False).apply(lambda x: x.ffill())
    
    df_full.dropna(subset=['winner'], inplace=True)
    print(f"Shape après imputation et suppression des nuls restants : {df_full.shape}")

    print("\nDonnées brutes chargées, imputées et prêtes pour le traitement.")
    display(df_full.head())
    
    if df_full.isnull().sum().sum() == 0:
        print("\n✅ Aucune valeur nulle détectée dans le jeu de données final.")
    else:
        print("\n❌ Attention : Des valeurs nulles sont toujours présentes.")
        display(df_full.isnull().sum())
except Exception as e:
    print(f"Erreur lors de la lecture des données depuis la vue : {e}")

Shape avant imputation : (752, 7)
Shape après imputation et suppression des nuls restants : (752, 7)

Données brutes chargées, imputées et prêtes pour le traitement.


/var/folders/sp/w9p124nd6jb9srzf94f0v2q80000gn/T/ipykernel_54991/2251613412.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_full = df_full.groupby('department_code', group_keys=False).apply(lambda x: x.ffill())


,year,department_code,winner,number_of_victims,immigration_rate,poverty_rate,unemployment_rate
0,2017,01,DROITE,20552,11.566667,10.5,6.750
1,2018,01,DROITE,21081,11.700000,10.3,6.300
2,2019,01,DROITE,21938,11.833333,10.7,6.050
3,2020,01,DROITE,19359,11.966667,10.5,6.075
4,2021,01,DROITE,21465,12.100000,10.8,5.925



✅ Aucune valeur nulle détectée dans le jeu de données final.


## 7. Division Temporelle (Train/Test)

In [90]:
train_df = df_full[df_full['year'] < 2024].copy()
test_df = df_full[df_full['year'] == 2024].copy()
train_df = train_df[train_df['winner'] != 'E.GAUCHE']
print(f"Taille du jeu d'entraînement (<= 2023) : {train_df.shape}")
print(f"Taille du jeu de test (2024) : {test_df.shape}")

Taille du jeu d'entraînement (<= 2023) : (652, 7)
Taille du jeu de test (2024) : (94, 7)


## 8. Définition des Features et de la Cible

In [91]:
X_train = train_df.drop(columns=['winner', 'year'])
y_train = train_df['winner']
X_test = test_df.drop(columns=['winner', 'year'])
y_test = test_df['winner']
print("Jeux de données X_train/y_train et X_test/y_test définis.")

Jeux de données X_train/y_train et X_test/y_test définis.


## 9. Pipeline de Pré-traitement

In [92]:
categorical_features = ['department_code']
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()

preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

label_encoder_y = LabelEncoder()
print("Pipeline de pré-traitement et encodeur de label définis.")

Pipeline de pré-traitement et encodeur de label définis.


## 10. Application des Transformations

In [93]:
X_train_processed = preprocessor_X.fit_transform(X_train)
X_test_processed = preprocessor_X.transform(X_test)
y_train_encoded = label_encoder_y.fit_transform(y_train)
y_test_encoded = label_encoder_y.transform(y_test)
print("Transformations appliquées aux données.")

Transformations appliquées aux données.


## 11. Sauvegarde des Données et des Transformateurs

### 11.1. Reconstruction et Sauvegarde des Données Traitées

In [94]:
try:
    ohe_feature_names = preprocessor_X.named_transformers_['cat'].get_feature_names_out(input_features=categorical_features)
    feature_names = numerical_features + list(ohe_feature_names)
    
    # Assurer que les noms des colonnes de features sont en minuscules
    feature_names = [f.lower() for f in feature_names]
    
    X_train_processed_df = pd.DataFrame(X_train_processed.toarray(), columns=feature_names)
    X_test_processed_df = pd.DataFrame(X_test_processed.toarray(), columns=feature_names)

    train_to_db = X_train_processed_df.copy()
    # Assurer que la colonne cible est en minuscules
    train_to_db['winner_encoded'] = y_train_encoded
    
    test_to_db = X_test_processed_df.copy()
    test_to_db['winner_encoded'] = y_test_encoded

    train_to_db.to_sql('processed_train_data', engine, if_exists='replace', index=False)
    test_to_db.to_sql('processed_test_data', engine, if_exists='replace', index=False)
    
    print(f"Tables sauvegardées dans PostgreSQL : 'processed_train_data' et 'processed_test_data'.")
    print(f"Colonnes sauvegardées : {list(train_to_db.columns)}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde des données traitées : {e}")

Tables sauvegardées dans PostgreSQL : 'processed_train_data' et 'processed_test_data'.
Colonnes sauvegardées : ['number_of_victims', 'immigration_rate', 'poverty_rate', 'unemployment_rate', 'department_code_01', 'department_code_02', 'department_code_03', 'department_code_04', 'department_code_05', 'department_code_06', 'department_code_07', 'department_code_08', 'department_code_09', 'department_code_10', 'department_code_11', 'department_code_12', 'department_code_13', 'department_code_14', 'department_code_15', 'department_code_16', 'department_code_17', 'department_code_18', 'department_code_19', 'department_code_21', 'department_code_22', 'department_code_23', 'department_code_24', 'department_code_25', 'department_code_26', 'department_code_27', 'department_code_28', 'department_code_29', 'department_code_30', 'department_code_31', 'department_code_32', 'department_code_33', 'department_code_34', 'department_code_35', 'department_code_36', 'department_code_37', 'department_code_3

### 11.2. Sauvegarde des Transformateurs

In [95]:
ARTIFACTS_DIR = '../database'
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
preprocessor_path = os.path.join(ARTIFACTS_DIR, 'preprocessor_X.joblib')
label_encoder_path = os.path.join(ARTIFACTS_DIR, 'label_encoder_y.joblib')
joblib.dump(preprocessor_X, preprocessor_path)
joblib.dump(label_encoder_y, label_encoder_path)
print(f"Préprocesseur sauvegardé : {preprocessor_path}")
print(f"Encodeur de label sauvegardé : {label_encoder_path}")

Préprocesseur sauvegardé : ../database/preprocessor_X.joblib
Encodeur de label sauvegardé : ../database/label_encoder_y.joblib


## 12. Fermeture de la Connexion

In [96]:
engine.dispose()
print("Connexion à la base de données fermée.")

Connexion à la base de données fermée.
